In [1]:
!pip install deep-translator beautifulsoup4


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
import os
import glob
from deep_translator import GoogleTranslator
from bs4 import BeautifulSoup, Comment
import time
import re
import json

language_ = "ja"

def clean_html_formatting(html_str):
    """
    HTML 포맷팅 정리 함수
    - DOCTYPE 하단 불필요한 'html' 텍스트 제거
    """
    doctype_match = re.search(r'<!DOCTYPE[^>]*>', html_str, re.IGNORECASE)
    doctype = doctype_match.group(0) if doctype_match else ''
    after_doctype = html_str[doctype_match.end():] if doctype_match else html_str
    after_doctype = re.sub(r'^\s*html\s*\n', '', after_doctype, flags=re.IGNORECASE)
    if doctype:
        cleaned_html = doctype + '\n' + after_doctype.lstrip()
    else:
        cleaned_html = after_doctype
    return cleaned_html

def translate_json_content(json_str, translator):
    """
    JSON 구조화된 데이터 내 텍스트 번역
    """
    try:
        data = json.loads(json_str)
        
        def translate_recursive(obj):
            if isinstance(obj, dict):
                for key, value in obj.items():
                    if isinstance(value, str) and value.strip():
                        # 영어가 아닌 텍스트만 번역
                        if not is_english_text(value):
                            try:
                                obj[key] = translator.translate(value)
                            except:
                                pass
                    elif isinstance(value, (dict, list)):
                        translate_recursive(value)
            elif isinstance(obj, list):
                for i, item in enumerate(obj):
                    if isinstance(item, str) and item.strip():
                        if not is_english_text(item):
                            try:
                                obj[i] = translator.translate(item)
                            except:
                                pass
                    elif isinstance(item, (dict, list)):
                        translate_recursive(item)
        
        translate_recursive(data)
        return json.dumps(data, ensure_ascii=False, indent=16)
    except:
        return json_str

def is_english_text(text):
    """
    텍스트가 영어인지 간단히 판단
    """
    if not text.strip():
        return True
    
    # URL, 이메일, 특수 코드는 번역하지 않음
    if any(pattern in text.lower() for pattern in ['http', 'www.', '@', '.com', '.html', 'gtag', 'dataLayer']):
        return True
    
    # 숫자만 있는 경우
    if text.strip().replace(' ', '').replace('-', '').replace('/', '').isdigit():
        return True
    
    # 한글이 포함된 경우 한국어로 판단
    if any('\uac00' <= char <= '\ud7af' for char in text):
        return False
    
    return True

def translate_html_complete(input_folder, output_folder, source_lang='auto', target_lang=language_):
    """
    HTML 파일들을 완전히 번역하는 함수 (메타태그, JSON 데이터 포함)
    """
    os.makedirs(output_folder, exist_ok=True)
    translator = GoogleTranslator(source=source_lang, target=target_lang)
    html_files = glob.glob(os.path.join(input_folder, '*.html'))
    
    if not html_files:
        print(f"'{input_folder}' 폴더에 HTML 파일이 없습니다.")
        return
    
    print(f"총 {len(html_files)}개의 HTML 파일을 발견했습니다.")
    
    for i, html_file in enumerate(html_files):
        filename = os.path.basename(html_file)
        output_file = os.path.join(output_folder, filename)
        
        if os.path.exists(output_file):
            print(f"⏭️  {filename} - 이미 번역되어 건너뜀")
            continue
        
        print(f"🔄 번역 중... ({i+1}/{len(html_files)}): {filename}")
        
        try:
            with open(html_file, 'r', encoding='utf-8') as f:
                original_content = f.read()
            
            # DOCTYPE 선언 추출 및 보존
            doctype_match = re.search(r'<!DOCTYPE[^>]*>', original_content, re.IGNORECASE)
            original_doctype = doctype_match.group(0) if doctype_match else None
            
            # BeautifulSoup으로 파싱
            soup = BeautifulSoup(original_content, 'html.parser')
            
            translated_count = 0
            
            # 1. 메타 태그 번역
            meta_tags = soup.find_all('meta')
            for meta in meta_tags:
                if meta.get('content') and not is_english_text(meta.get('content')):
                    try:
                        translated_content = translator.translate(meta.get('content'))
                        meta['content'] = translated_content
                        translated_count += 1
                        print(f"   🏷️  메타태그 번역: {meta.get('name') or meta.get('property')}")
                    except:
                        pass
            
            # 2. JSON-LD 구조화된 데이터 번역
            json_scripts = soup.find_all('script', type='application/ld+json')
            for script in json_scripts:
                if script.string:
                    translated_json = translate_json_content(script.string, translator)
                    script.string = translated_json
                    print(f"   📊 JSON-LD 데이터 번역 완료")
            
            # 3. JavaScript 주석 및 문자열 번역
            js_scripts = soup.find_all('script', type=lambda x: x != 'application/ld+json')
            for script in js_scripts:
                if script.string:
                    js_content = script.string
                    # JavaScript 주석 내 한국어 번역
                    comment_pattern = r'//\s*(.+)'
                    matches = re.findall(comment_pattern, js_content)
                    for match in matches:
                        if not is_english_text(match):
                            try:
                                translated_comment = translator.translate(match)
                                js_content = js_content.replace(match, translated_comment)
                                translated_count += 1
                            except:
                                pass
                    
                    # JavaScript 문자열 내 한국어 번역
                    string_pattern = r'"([^"]*[가-힣]+[^"]*)"'
                    matches = re.findall(string_pattern, js_content)
                    for match in matches:
                        if not is_english_text(match):
                            try:
                                translated_string = translator.translate(match)
                                js_content = js_content.replace(f'"{match}"', f'"{translated_string}"')
                                translated_count += 1
                            except:
                                pass
                    
                    script.string = js_content
            
            # 4. 일반 텍스트 노드 번역
            text_nodes = soup.find_all(text=True)
            for text in text_nodes:
                if isinstance(text, Comment):
                    print(f"   💬 주석 건너뜀: {str(text).strip()[:30]}...")
                    continue
                
                if (text.strip() and 
                    text.parent.name not in ['script', 'style', 'meta'] and
                    not isinstance(text, Comment) and
                    not is_english_text(text.strip())):
                    try:
                        translated = translator.translate(text.strip())
                        text.replace_with(translated)
                        translated_count += 1
                    except Exception as e:
                        print(f"   ⚠️  텍스트 번역 실패: {text.strip()[:30]}... - {e}")
                        continue
            
            # 5. 속성값 번역 (alt, title, placeholder 등)
            for tag in soup.find_all():
                for attr in ['alt', 'title', 'placeholder', 'aria-label']:
                    if tag.get(attr) and not is_english_text(tag.get(attr)):
                        try:
                            translated_attr = translator.translate(tag.get(attr))
                            tag[attr] = translated_attr
                            translated_count += 1
                            print(f"   🔖 속성 번역: {attr}")
                        except:
                            pass
            
            # HTML 문자열로 변환
            translated_html = str(soup)
            
            # 원본 DOCTYPE 복원
            if original_doctype:
                translated_html = re.sub(r'<!DOCTYPE[^>]*>\s*', '', translated_html, flags=re.IGNORECASE)
                translated_html = original_doctype + '\n' + translated_html
                print(f"   📄 DOCTYPE 보존: {original_doctype}")
            
            # HTML 포맷팅 정리
            translated_html = clean_html_formatting(translated_html)
            
            # 번역된 파일 저장
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(translated_html)
            
            print(f"   ✅ 완료! ({translated_count}개 요소 번역됨)")
            
            # API 제한 방지를 위한 딜레이
            time.sleep(1)
            
        except Exception as e:
            print(f"   ❌ 오류 발생: {filename} - {e}")
    
    print("\n🎉 모든 파일 완전 번역 완료!")

def main():
    """
    사용자가 폴더 경로를 입력하여 완전 번역을 실행하는 함수
    """
    print("=== HTML 폴더 완전 번역 프로그램 ===\n")
    print("📋 번역 대상:")
    print("  • HTML 텍스트 콘텐츠")
    print("  • 메타 태그 (description, keywords, og:title 등)")
    print("  • JSON-LD 구조화된 데이터")
    print("  • JavaScript 주석 및 문자열")
    print("  • 속성값 (alt, title, placeholder 등)")
    print()
    
    input_folder = input("번역할 HTML 파일들이 있는 폴더 경로를 입력하세요: ").strip()
    output_folder = input("번역된 파일들을 저장할 폴더 경로를 입력하세요: ").strip()
    
    if not os.path.exists(input_folder):
        print(f"❌ 입력 폴더가 존재하지 않습니다: {input_folder}")
        return
    
    translate_html_complete(input_folder, output_folder)

if __name__ == "__main__":
    main()


=== HTML 폴더 완전 번역 프로그램 ===

📋 번역 대상:
  • HTML 텍스트 콘텐츠
  • 메타 태그 (description, keywords, og:title 등)
  • JSON-LD 구조화된 데이터
  • JavaScript 주석 및 문자열
  • 속성값 (alt, title, placeholder 등)



번역할 HTML 파일들이 있는 폴더 경로를 입력하세요:  C:\Users\User\Documents\GitHub\emfls.github.io\report\travel
번역된 파일들을 저장할 폴더 경로를 입력하세요:  C:\Users\User\Downloads\perplexity_results


총 5385개의 HTML 파일을 발견했습니다.
⏭️  albania-berat.html - 이미 번역되어 건너뜀
⏭️  albania-durres.html - 이미 번역되어 건너뜀
⏭️  albania-elbasan.html - 이미 번역되어 건너뜀
⏭️  albania-fier.html - 이미 번역되어 건너뜀
⏭️  albania-gjirokaster.html - 이미 번역되어 건너뜀
⏭️  albania-kavaje.html - 이미 번역되어 건너뜀
⏭️  albania-korce.html - 이미 번역되어 건너뜀
⏭️  albania-kruje.html - 이미 번역되어 건너뜀
⏭️  albania-kucove.html - 이미 번역되어 건너뜀
⏭️  albania-kukes.html - 이미 번역되어 건너뜀
⏭️  albania-lac.html - 이미 번역되어 건너뜀
⏭️  albania-lezhe.html - 이미 번역되어 건너뜀
⏭️  albania-lushnje.html - 이미 번역되어 건너뜀
⏭️  albania-patos.html - 이미 번역되어 건너뜀
⏭️  albania-peshkopi.html - 이미 번역되어 건너뜀
⏭️  albania-pogradec.html - 이미 번역되어 건너뜀
⏭️  albania-saranda.html - 이미 번역되어 건너뜀
⏭️  albania-shkoder.html - 이미 번역되어 건너뜀
⏭️  albania-tirana.html - 이미 번역되어 건너뜀
⏭️  albania-vlore.html - 이미 번역되어 건너뜀
⏭️  algeria-adrar.html - 이미 번역되어 건너뜀
⏭️  algeria-ain-temouchent.html - 이미 번역되어 건너뜀
⏭️  algeria-aindefla.html - 이미 번역되어 건너뜀
⏭️  algeria-algiers.html - 이미 번역되어 건너뜀
⏭️  algeria-annaba.html - 이미 번역되어 건너뜀
⏭️  algeria-b

C:\Users\User\AppData\Local\Temp\ipykernel_4024\3475159143.py:169: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_nodes = soup.find_all(text=True)


   💬 주석 건너뜀: Open Graph Meta Tags...
   💬 주석 건너뜀: Canonical URL...
   💬 주석 건너뜀: Google 태그 (gtag.js)...
   💬 주석 건너뜀: Structured Data...
   📄 DOCTYPE 보존: <!doctype html>
   ✅ 완료! (182개 요소 번역됨)
🔄 번역 중... (69/5385): angola-bie.html
   🏷️  메타태그 번역: description
   🏷️  메타태그 번역: keywords
   🏷️  메타태그 번역: og:title
   🏷️  메타태그 번역: og:description
   📊 JSON-LD 데이터 번역 완료
   💬 주석 건너뜀: Open Graph Meta Tags...
   💬 주석 건너뜀: Canonical URL...
   💬 주석 건너뜀: Google 태그 (gtag.js)...
   💬 주석 건너뜀: Structured Data...
   📄 DOCTYPE 보존: <!doctype html>
   ✅ 완료! (172개 요소 번역됨)
🔄 번역 중... (70/5385): angola-cabinda.html
   🏷️  메타태그 번역: description
   🏷️  메타태그 번역: keywords
   🏷️  메타태그 번역: og:title
   🏷️  메타태그 번역: og:description
   📊 JSON-LD 데이터 번역 완료
   💬 주석 건너뜀: Open Graph Meta Tags...
   💬 주석 건너뜀: Canonical URL...
   💬 주석 건너뜀: Google 태그 (gtag.js)...
   💬 주석 건너뜀: Structured Data...
   📄 DOCTYPE 보존: <!doctype html>
   ✅ 완료! (203개 요소 번역됨)
🔄 번역 중... (71/5385): angola-cacuso.html
   🏷️  메타태그 번역: description
   🏷️  메타태그 번역: k

KeyboardInterrupt: 

In [ ]:
import os
import glob
from deep_translator import GoogleTranslator
from bs4 import BeautifulSoup, Comment
import time
import re
import json
import concurrent.futures
import threading

def clean_html_formatting(html_str):
    """
    HTML 포맷팅 정리 함수
    - DOCTYPE 하단 불필요한 'html' 텍스트 제거
    """
    doctype_match = re.search(r'<!DOCTYPE[^>]*>', html_str, re.IGNORECASE)
    doctype = doctype_match.group(0) if doctype_match else ''
    after_doctype = html_str[doctype_match.end():] if doctype_match else html_str
    after_doctype = re.sub(r'^\s*html\s*\n', '', after_doctype, flags=re.IGNORECASE)
    if doctype:
        cleaned_html = doctype + '\n' + after_doctype.lstrip()
    else:
        cleaned_html = after_doctype
    return cleaned_html

def translate_json_content(json_str, translator):
    """
    JSON 구조화된 데이터 내 텍스트 번역
    """
    try:
        data = json.loads(json_str)
        
        def translate_recursive(obj):
            if isinstance(obj, dict):
                for key, value in obj.items():
                    if isinstance(value, str) and value.strip():
                        # 영어가 아닌 텍스트만 번역
                        if not is_english_text(value):
                            try:
                                obj[key] = translator.translate(value)
                            except:
                                pass
                    elif isinstance(value, (dict, list)):
                        translate_recursive(value)
            elif isinstance(obj, list):
                for i, item in enumerate(obj):
                    if isinstance(item, str) and item.strip():
                        if not is_english_text(item):
                            try:
                                obj[i] = translator.translate(item)
                            except:
                                pass
                    elif isinstance(item, (dict, list)):
                        translate_recursive(item)
        
        translate_recursive(data)
        return json.dumps(data, ensure_ascii=False, indent=16)
    except:
        return json_str

def is_english_text(text):
    """
    텍스트가 영어인지 간단히 판단
    """
    if not text.strip():
        return True
    
    # URL, 이메일, 특수 코드는 번역하지 않음
    if any(pattern in text.lower() for pattern in ['http', 'www.', '@', '.com', '.html', 'gtag', 'dataLayer']):
        return True
    
    # 숫자만 있는 경우
    if text.strip().replace(' ', '').replace('-', '').replace('/', '').isdigit():
        return True
    
    # 한글이 포함된 경우 한국어로 판단
    if any('\uac00' <= char <= '\ud7af' for char in text):
        return False
    
    return True

def translate_single_html_file(html_file, output_folder, request_lock, last_request_time, delay_between_requests=2):
    """
    단일 HTML 파일을 번역하는 함수 (병렬 처리용)
    """
    filename = os.path.basename(html_file)
    output_file = os.path.join(output_folder, filename)
    
    if os.path.exists(output_file):
        print(f"⏭️  {filename} - 이미 번역되어 건너뜀")
        return
    
    # API 요청 간격 조절
    with request_lock:
        current_time = time.time()
        time_since_last = current_time - last_request_time[0]
        if time_since_last < delay_between_requests:
            time.sleep(delay_between_requests - time_since_last)
        last_request_time[0] = time.time()
    
    print(f"🔄 번역 중... {filename}")
    
    try:
        # 각 스레드마다 별도의 translator 인스턴스 생성
        translator = GoogleTranslator(source='auto', target=language_)
        
        with open(html_file, 'r', encoding='utf-8') as f:
            original_content = f.read()
        
        # DOCTYPE 선언 추출 및 보존
        doctype_match = re.search(r'<!DOCTYPE[^>]*>', original_content, re.IGNORECASE)
        original_doctype = doctype_match.group(0) if doctype_match else None
        
        # BeautifulSoup으로 파싱
        soup = BeautifulSoup(original_content, 'html.parser')
        
        translated_count = 0
        
        # 1. 메타 태그 번역
        meta_tags = soup.find_all('meta')
        for meta in meta_tags:
            if meta.get('content') and not is_english_text(meta.get('content')):
                try:
                    translated_content = translator.translate(meta.get('content'))
                    meta['content'] = translated_content
                    translated_count += 1
                    print(f"   🏷️  메타태그 번역: {meta.get('name') or meta.get('property')}")
                except:
                    pass
        
        # 2. JSON-LD 구조화된 데이터 번역
        json_scripts = soup.find_all('script', type='application/ld+json')
        for script in json_scripts:
            if script.string:
                translated_json = translate_json_content(script.string, translator)
                script.string = translated_json
                print(f"   📊 JSON-LD 데이터 번역 완료")
        
        # 3. JavaScript 주석 및 문자열 번역
        js_scripts = soup.find_all('script', type=lambda x: x != 'application/ld+json')
        for script in js_scripts:
            if script.string:
                js_content = script.string
                # JavaScript 주석 내 한국어 번역
                comment_pattern = r'//\s*(.+)'
                matches = re.findall(comment_pattern, js_content)
                for match in matches:
                    if not is_english_text(match):
                        try:
                            translated_comment = translator.translate(match)
                            js_content = js_content.replace(match, translated_comment)
                            translated_count += 1
                        except:
                            pass
                
                # JavaScript 문자열 내 한국어 번역
                string_pattern = r'"([^"]*[가-힣]+[^"]*)"'
                matches = re.findall(string_pattern, js_content)
                for match in matches:
                    if not is_english_text(match):
                        try:
                            translated_string = translator.translate(match)
                            js_content = js_content.replace(f'"{match}"', f'"{translated_string}"')
                            translated_count += 1
                        except:
                            pass
                
                script.string = js_content
        
        # 4. 일반 텍스트 노드 번역
        text_nodes = soup.find_all(text=True)
        for text in text_nodes:
            if isinstance(text, Comment):
                continue
            
            if (text.strip() and 
                text.parent.name not in ['script', 'style', 'meta'] and
                not isinstance(text, Comment) and
                not is_english_text(text.strip())):
                try:
                    translated = translator.translate(text.strip())
                    text.replace_with(translated)
                    translated_count += 1
                except Exception as e:
                    print(f"   ⚠️  텍스트 번역 실패: {text.strip()[:30]}... - {e}")
                    continue
        
        # 5. 속성값 번역 (alt, title, placeholder 등)
        for tag in soup.find_all():
            for attr in ['alt', 'title', 'placeholder', 'aria-label']:
                if tag.get(attr) and not is_english_text(tag.get(attr)):
                    try:
                        translated_attr = translator.translate(tag.get(attr))
                        tag[attr] = translated_attr
                        translated_count += 1
                        print(f"   🔖 속성 번역: {attr}")
                    except:
                        pass
        
        # HTML 문자열로 변환
        translated_html = str(soup)
        
        # 원본 DOCTYPE 복원
        if original_doctype:
            translated_html = re.sub(r'<!DOCTYPE[^>]*>\s*', '', translated_html, flags=re.IGNORECASE)
            translated_html = original_doctype + '\n' + translated_html
        
        # HTML 포맷팅 정리
        translated_html = clean_html_formatting(translated_html)
        
        # 번역된 파일 저장
        with open(output_file, 'w', encoding='utf-8') as f:
            f.write(translated_html)
        
        print(f"   ✅ 완료! {filename} ({translated_count}개 요소 번역됨)")
        
    except Exception as e:
        print(f"   ❌ 오류 발생: {filename} - {e}")

def translate_html_parallel(input_folder, output_folder, max_workers=3, delay_between_requests=2, source_lang='auto', target_lang=language_):
    """
    병렬 처리로 HTML 파일들을 번역하는 함수
    """
    os.makedirs(output_folder, exist_ok=True)
    html_files = glob.glob(os.path.join(input_folder, '*.html'))
    
    if not html_files:
        print(f"'{input_folder}' 폴더에 HTML 파일이 없습니다.")
        return
    
    print(f"총 {len(html_files)}개의 HTML 파일을 {max_workers}개 스레드로 처리합니다.")
    print(f"API 요청 간격: {delay_between_requests}초")
    print()
    
    # 스레드 간 요청 간격 조절을 위한 락
    request_lock = threading.Lock()
    last_request_time = [0]  # 리스트로 감싸서 참조 가능하게 함
    
    # ThreadPoolExecutor로 병렬 처리
    with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = [
            executor.submit(
                translate_single_html_file, 
                html_file, 
                output_folder, 
                request_lock, 
                last_request_time, 
                delay_between_requests
            ) 
            for html_file in html_files
        ]
        
        # 모든 작업 완료 대기
        completed = 0
        for future in concurrent.futures.as_completed(futures):
            try:
                future.result()
                completed += 1
                print(f"📊 진행률: {completed}/{len(html_files)} 완료")
            except Exception as e:
                print(f"스레드 실행 중 오류: {e}")
    
    print("\n🎉 모든 파일 병렬 번역 완료!")

def translate_html_complete(input_folder, output_folder, source_lang='auto', target_lang=language_):
    """
    HTML 파일들을 순차적으로 번역하는 함수 (기존 방식)
    """
    os.makedirs(output_folder, exist_ok=True)
    translator = GoogleTranslator(source=source_lang, target=target_lang)
    html_files = glob.glob(os.path.join(input_folder, '*.html'))
    
    if not html_files:
        print(f"'{input_folder}' 폴더에 HTML 파일이 없습니다.")
        return
    
    print(f"총 {len(html_files)}개의 HTML 파일을 발견했습니다.")
    
    for i, html_file in enumerate(html_files):
        filename = os.path.basename(html_file)
        output_file = os.path.join(output_folder, filename)
        
        if os.path.exists(output_file):
            print(f"⏭️  {filename} - 이미 번역되어 건너뜀")
            continue
        
        print(f"🔄 번역 중... ({i+1}/{len(html_files)}): {filename}")
        
        try:
            with open(html_file, 'r', encoding='utf-8') as f:
                original_content = f.read()
            
            # DOCTYPE 선언 추출 및 보존
            doctype_match = re.search(r'<!DOCTYPE[^>]*>', original_content, re.IGNORECASE)
            original_doctype = doctype_match.group(0) if doctype_match else None
            
            # BeautifulSoup으로 파싱
            soup = BeautifulSoup(original_content, 'html.parser')
            
            translated_count = 0
            
            # 1. 메타 태그 번역
            meta_tags = soup.find_all('meta')
            for meta in meta_tags:
                if meta.get('content') and not is_english_text(meta.get('content')):
                    try:
                        translated_content = translator.translate(meta.get('content'))
                        meta['content'] = translated_content
                        translated_count += 1
                        print(f"   🏷️  메타태그 번역: {meta.get('name') or meta.get('property')}")
                    except:
                        pass
            
            # 2. JSON-LD 구조화된 데이터 번역
            json_scripts = soup.find_all('script', type='application/ld+json')
            for script in json_scripts:
                if script.string:
                    translated_json = translate_json_content(script.string, translator)
                    script.string = translated_json
                    print(f"   📊 JSON-LD 데이터 번역 완료")
            
            # 3. JavaScript 주석 및 문자열 번역
            js_scripts = soup.find_all('script', type=lambda x: x != 'application/ld+json')
            for script in js_scripts:
                if script.string:
                    js_content = script.string
                    # JavaScript 주석 내 한국어 번역
                    comment_pattern = r'//\s*(.+)'
                    matches = re.findall(comment_pattern, js_content)
                    for match in matches:
                        if not is_english_text(match):
                            try:
                                translated_comment = translator.translate(match)
                                js_content = js_content.replace(match, translated_comment)
                                translated_count += 1
                            except:
                                pass
                    
                    # JavaScript 문자열 내 한국어 번역
                    string_pattern = r'"([^"]*[가-힣]+[^"]*)"'
                    matches = re.findall(string_pattern, js_content)
                    for match in matches:
                        if not is_english_text(match):
                            try:
                                translated_string = translator.translate(match)
                                js_content = js_content.replace(f'"{match}"', f'"{translated_string}"')
                                translated_count += 1
                            except:
                                pass
                    
                    script.string = js_content
            
            # 4. 일반 텍스트 노드 번역
            text_nodes = soup.find_all(text=True)
            for text in text_nodes:
                if isinstance(text, Comment):
                    print(f"   💬 주석 건너뜀: {str(text).strip()[:30]}...")
                    continue
                
                if (text.strip() and 
                    text.parent.name not in ['script', 'style', 'meta'] and
                    not isinstance(text, Comment) and
                    not is_english_text(text.strip())):
                    try:
                        translated = translator.translate(text.strip())
                        text.replace_with(translated)
                        translated_count += 1
                    except Exception as e:
                        print(f"   ⚠️  텍스트 번역 실패: {text.strip()[:30]}... - {e}")
                        continue
            
            # 5. 속성값 번역 (alt, title, placeholder 등)
            for tag in soup.find_all():
                for attr in ['alt', 'title', 'placeholder', 'aria-label']:
                    if tag.get(attr) and not is_english_text(tag.get(attr)):
                        try:
                            translated_attr = translator.translate(tag.get(attr))
                            tag[attr] = translated_attr
                            translated_count += 1
                            print(f"   🔖 속성 번역: {attr}")
                        except:
                            pass
            
            # HTML 문자열로 변환
            translated_html = str(soup)
            
            # 원본 DOCTYPE 복원
            if original_doctype:
                translated_html = re.sub(r'<!DOCTYPE[^>]*>\s*', '', translated_html, flags=re.IGNORECASE)
                translated_html = original_doctype + '\n' + translated_html
                print(f"   📄 DOCTYPE 보존: {original_doctype}")
            
            # HTML 포맷팅 정리
            translated_html = clean_html_formatting(translated_html)
            
            # 번역된 파일 저장
            with open(output_file, 'w', encoding='utf-8') as f:
                f.write(translated_html)
            
            print(f"   ✅ 완료! ({translated_count}개 요소 번역됨)")
            
            # API 제한 방지를 위한 딜레이
            time.sleep(1)
            
        except Exception as e:
            print(f"   ❌ 오류 발생: {filename} - {e}")
    
    print("\n🎉 모든 파일 완전 번역 완료!")

def main():
    """
    사용자가 폴더 경로를 입력하여 완전 번역을 실행하는 함수
    """
    print("=== HTML 폴더 완전 번역 프로그램 ===\n")
    print("📋 번역 대상:")
    print("  • HTML 텍스트 콘텐츠")
    print("  • 메타 태그 (description, keywords, og:title 등)")
    print("  • JSON-LD 구조화된 데이터")
    print("  • JavaScript 주석 및 문자열")
    print("  • 속성값 (alt, title, placeholder 등)")
    print()
    
    input_folder = input("번역할 HTML 파일들이 있는 폴더 경로를 입력하세요: ").strip()
    output_folder = input("번역된 파일들을 저장할 폴더 경로를 입력하세요: ").strip()
    
    if not os.path.exists(input_folder):
        print(f"❌ 입력 폴더가 존재하지 않습니다: {input_folder}")
        return
    
    # 처리 방식 선택
    print("\n처리 방식을 선택하세요:")
    print("1. 순차 처리 (안전하지만 느림)")
    print("2. 병렬 처리 (빠르지만 API 제한 주의)")
    
    choice = input("선택 (1 또는 2): ").strip()
    
    if choice == "2":
        max_workers = 3
        delay = 3
        translate_html_parallel(input_folder, output_folder, max_workers=max_workers, delay_between_requests=delay)
    else:
        translate_html_complete(input_folder, output_folder)

if __name__ == "__main__":
    main()


=== HTML 폴더 완전 번역 프로그램 ===

📋 번역 대상:
  • HTML 텍스트 콘텐츠
  • 메타 태그 (description, keywords, og:title 등)
  • JSON-LD 구조화된 데이터
  • JavaScript 주석 및 문자열
  • 속성값 (alt, title, placeholder 등)



번역할 HTML 파일들이 있는 폴더 경로를 입력하세요:  C:\Users\User\Documents\GitHub\emfls.github.io\report\travel
번역된 파일들을 저장할 폴더 경로를 입력하세요:  C:\Users\User\Downloads\perplexity_results



처리 방식을 선택하세요:
1. 순차 처리 (안전하지만 느림)
2. 병렬 처리 (빠르지만 API 제한 주의)


선택 (1 또는 2):  2
동시 처리할 파일 수 (권장: 3-5):  3
API 요청 간격 (초, 권장: 2-3):  3


총 5385개의 HTML 파일을 3개 스레드로 처리합니다.
API 요청 간격: 3초

⏭️  albania-berat.html - 이미 번역되어 건너뜀
⏭️  albania-durres.html - 이미 번역되어 건너뜀
⏭️  albania-elbasan.html - 이미 번역되어 건너뜀
⏭️  albania-gjirokaster.html - 이미 번역되어 건너뜀
⏭️  albania-fier.html - 이미 번역되어 건너뜀
⏭️  albania-kavaje.html - 이미 번역되어 건너뜀
⏭️  albania-kucove.html - 이미 번역되어 건너뜀
⏭️  albania-korce.html - 이미 번역되어 건너뜀⏭️  albania-kruje.html - 이미 번역되어 건너뜀

⏭️  albania-kukes.html - 이미 번역되어 건너뜀
⏭️  albania-lezhe.html - 이미 번역되어 건너뜀
⏭️  albania-lac.html - 이미 번역되어 건너뜀
⏭️  albania-lushnje.html - 이미 번역되어 건너뜀
⏭️  albania-peshkopi.html - 이미 번역되어 건너뜀
⏭️  albania-pogradec.html - 이미 번역되어 건너뜀
⏭️  albania-saranda.html - 이미 번역되어 건너뜀
⏭️  albania-shkoder.html - 이미 번역되어 건너뜀
⏭️  albania-patos.html - 이미 번역되어 건너뜀
⏭️  albania-tirana.html - 이미 번역되어 건너뜀
⏭️  algeria-adrar.html - 이미 번역되어 건너뜀
⏭️  albania-vlore.html - 이미 번역되어 건너뜀
⏭️  algeria-ain-temouchent.html - 이미 번역되어 건너뜀
⏭️  algeria-aindefla.html - 이미 번역되어 건너뜀
⏭️  algeria-algiers.html - 이미 번역되어 건너뜀
⏭️  algeria-batna.html - 이미 번

C:\Users\User\AppData\Local\Temp\ipykernel_4024\3275510111.py:171: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text_nodes = soup.find_all(text=True)


   🏷️  메타태그 번역: og:description
🔄 번역 중... macao-coloane.html
   🏷️  메타태그 번역: description
   📊 JSON-LD 데이터 번역 완료
   🏷️  메타태그 번역: keywords
   🏷️  메타태그 번역: og:title
   🏷️  메타태그 번역: og:description
   📊 JSON-LD 데이터 번역 완료
   ✅ 완료! luxembourg-wissen.html (171개 요소 번역됨)
🔄 번역 중... macau-cotai.html
📊 진행률: 2916/5385 완료
   🏷️  메타태그 번역: description
   ✅ 완료! luxembourg-wasserbillig.html (171개 요소 번역됨)
📊 진행률: 2917/5385 완료
   🏷️  메타태그 번역: keywords
   🏷️  메타태그 번역: og:title
🔄 번역 중... macau-fishermans-wharf.html
   🏷️  메타태그 번역: og:description
   🏷️  메타태그 번역: description
   🏷️  메타태그 번역: keywords
   🏷️  메타태그 번역: og:title
   🏷️  메타태그 번역: og:description
   📊 JSON-LD 데이터 번역 완료
   📊 JSON-LD 데이터 번역 완료
   ✅ 완료! macao-coloane.html (173개 요소 번역됨)
📊 진행률: 2918/5385 완료
🔄 번역 중... macau-peninsula.html
   🏷️  메타태그 번역: description
   🏷️  메타태그 번역: keywords
   🏷️  메타태그 번역: og:title
   🏷️  메타태그 번역: og:description
   📊 JSON-LD 데이터 번역 완료
   ✅ 완료! macau-fishermans-wharf.html (173개 요소 번역됨)
📊 진행률: 2919/5385 완료
🔄 번역 중... macau-taipa.